In [1]:
pip install spacy-langdetect

     |████████████████████████████████| 998 kB 30.8 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=58469d7b6bf3d4186cd88bf066f7f876634f19b2462c26f3645242d566bfd19c
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [2]:
#Import libraries
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")
import re
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import unicodedata
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [3]:
#load stop words
file1 = open("/content/list.txt")
 
# Use this to read file content as a stream:
line = file1.read()
stop_words = line.split()

In [4]:
#Text Cleaning Final
def remove_punctuation(text):
  return re.sub(r'[^\w\s]+','',text)

def remove_english_and_numbers(text):
  return re.sub(r'[a-zA-Z\d+]+','',text)

def remove_escape_characters(text):
  text=re.sub(r'\n', '',text)
  text=re.sub(r'\_','',text)
  return text

def remove_non_arabic_non_ascii(text):
  nlp = spacy.load('en')  # 1
  nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
  text_content = text
  doc = nlp(text_content) #3
  detect_language = doc._.language #4
  if detect_language['language']=='ar':
    pass
  else:
      text=unicodedata.normalize('NFKD',text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      text=remove_punctuation(text)
      text=remove_english_and_numbers(text)
  return text

def text2words(text):
   return text.split()

def remove_stopwords(words, stop_words):
  return [word for word in words if word not in stop_words]

def normalize_text(text):
    text=remove_punctuation(text)
    text=remove_english_and_numbers(text)
    text=remove_escape_characters(text)
    text=remove_non_arabic_non_ascii(text)
    words=text2words(text)
    words = remove_stopwords(words, stop_words)
    text=' '.join(words)
    return text

In [5]:
#Load dataset
from sklearn.utils import shuffle
df=shuffle(pd.read_csv("/content/dataset_36000_2000.csv"))

In [6]:
df.head()

,Unnamed: 0,0,dialect
5807,5807,@afq2022 وينك يا بو فهد ، توني متصل لك جوالك م...,QA
27643,27643,والاحلى من الفرش .. الاسفنجه الي تكبر بالماي\n...,KW
12751,12751,@RMSumrain94 واحد مي لو سمحتي,JO
25046,25046,@Walo2a__ ماراح احكي ولا قالا شي هي حرا تعمل ت...,LB
5016,5016,@uae_3G @saoud_k_althani بتدفعون فعلتكم الشينه...,QA


In [7]:
df.drop(['Unnamed: 0'], axis=1,inplace=True)
df.rename(columns={'0': 'text'},inplace=True)
df=df.reset_index(drop=True)
df.head()

,text,dialect
0,@afq2022 وينك يا بو فهد ، توني متصل لك جوالك م...,QA
1,والاحلى من الفرش .. الاسفنجه الي تكبر بالماي\n...,KW
2,@RMSumrain94 واحد مي لو سمحتي,JO
3,@Walo2a__ ماراح احكي ولا قالا شي هي حرا تعمل ت...,LB
4,@uae_3G @saoud_k_althani بتدفعون فعلتكم الشينه...,QA


In [2]:
for i in range(len(df['text'])):
  df.text[i]=normalize_text(df.text[i])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     18000 non-null  object
 1   dialect  18000 non-null  object
dtypes: object(2)
memory usage: 281.4+ KB


In [ ]:
df= df.replace(r'^\s*$', np.NaN, regex=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     12505 non-null  object
 1   dialect  18000 non-null  object
dtypes: object(2)
memory usage: 281.4+ KB


In [ ]:
df.dropna(subset = ["text"], inplace=True)
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12505 entries, 0 to 12504
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     12505 non-null  object
 1   dialect  12505 non-null  object
dtypes: object(2)
memory usage: 195.5+ KB


In [ ]:
# Selecting duplicate rows based
# on 'text' column
duplicate = df[df.duplicated('text')]
 
print("Duplicate Rows based on text :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows based on text :


,text,dialect
6370,العسل والورد للحلوين,JO
6938,العمر المديد والعيش السعيد,PL
12038,تستاهل خير ياغالي,SA


In [ ]:
df.to_csv('cleaned_text_18000_1000.csv')

In [ ]:
df['dialect'].value_counts()

SY    945
EG    852
YE    831
SD    807
LY    727
IQ    717
LB    701
PL    699
OM    694
KW    693
TN    670
DZ    628
QA    623
SA    617
BH    607
JO    583
AE    572
MA    539
Name: dialect, dtype: int64

In [ ]:
#Create new dataframe containing only 100 tweet from each dialect
new_df= pd.DataFrame()
for x in df["dialect"].unique():
    new_df=new_df.append(df.loc[df['dialect'] == x][:539],ignore_index=True)

In [ ]:
new_df['dialect'].value_counts()

DZ    539
MA    539
BH    539
PL    539
SA    539
SD    539
LB    539
YE    539
OM    539
QA    539
KW    539
EG    539
AE    539
SY    539
LY    539
IQ    539
JO    539
TN    539
Name: dialect, dtype: int64

In [ ]:
new_df.to_csv('cleaned_539.csv')